<a href="https://colab.research.google.com/github/BMClab/BMC/blob/master/sistema_coordenadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Determinação de sistemas de coordenadas anatômicos com dados experimentais

> Cálculo dos sistemas de coordenadas anatômicos usando a convenção [VAKHUM](https://github.com/BMClab/BMC/blob/master/courses/refs/VAKHUM.pdf).

In [ ]:
#@title ## Setup {display-mode: "form"}

import pandas as pd
import numpy as np
# Function CCS plots Cartesian coordinate system and data
import plotly.graph_objects as go
import plotly.express as px

def plot_arrow_3d(start_point, end_point, color='blue', arrow_length_ratio=0.1, cone_size=0.05):
    """
    Plots a 3D arrow from start_point to end_point using Plotly.

    Args:
        start_point (np.array): The starting point of the arrow (x, y, z).
        end_point (np.array): The ending point of the arrow (x, y, z).
        color (str): The color of the arrow.
        arrow_length_ratio (float): The ratio of the arrowhead length to the total arrow length.
        cone_size (float): The size of the cone representing the arrowhead.

    Returns:
        list: A list of Plotly graph objects representing the arrow.
    """
    # Draw the line segment
    line_trace = go.Scatter3d(
        x=[start_point[0], end_point[0]],
        y=[start_point[1], end_point[1]],
        z=[start_point[2], end_point[2]],
        mode='lines',
        line=dict(color=color, width=4),
        showlegend=False
    )

    # Calculate direction vector
    direction = end_point - start_point
    arrow_length = np.linalg.norm(direction)

    if arrow_length > 0:
        direction_normalized = direction / arrow_length

        # Add a cone at the endpoint for the arrowhead
        cone_base = end_point - direction_normalized * (arrow_length * arrow_length_ratio)
        cone_trace = go.Cone(
            x=[end_point[0]],
            y=[end_point[1]],
            z=[end_point[2]],
            u=[direction_normalized[0]],
            v=[direction_normalized[1]],
            w=[direction_normalized[2]],
            sizemode="absolute",
            sizeref=cone_size,
            colorscale=[[0, color], [1, color]],
            showscale=False,
            showlegend=False
        )
        return [line_trace, cone_trace]
    else:
        # If start and end points are the same, just return the line trace (or an empty list)
        return [line_trace]


def plot_ccs(Oijk=[0, 0, 0], ijk=[], ijk_label=True,
             Oxyz=[0, 0, 0], xyz=[], xyz_label=True, point=[],
             point_label=None, proj_lines=True, vector=True):
    """
    Plots a Cartesian coordinate system and data using Plotly.

    Args:
        Oijk (list): Origin of the ijk basis.
        ijk (list): Basis vectors for the ijk system.
        ijk_label (bool): Whether to label the ijk axes.
        Oxyz (list): Origin of the xyz basis.
        xyz (list): Basis vectors for the xyz system.
        xyz_label (bool): Whether to label the xyz axes.
        point (list): List of points to plot.
        point_label (list): List of labels for the points.
        proj_lines (bool): Whether to draw projection lines from points to axes.
        vector (bool): Whether to draw vectors from the origin to the points.

    Returns:
        go.Figure: The Plotly figure object.
    """
    fig = go.Figure()

    x0, y0, z0 = Oxyz
    if len(xyz):
        xyz = np.asarray(xyz)
        x, y, z = xyz
        # X-axis
        fig.add_traces(plot_arrow_3d(np.array(Oxyz), np.array(Oxyz) + x, color='black', cone_size=0.1, arrow_length_ratio=0.05))
        # Y-axis
        fig.add_traces(plot_arrow_3d(np.array(Oxyz), np.array(Oxyz) + y, color='black', cone_size=0.1, arrow_length_ratio=0.05))
        # Z-axis
        fig.add_traces(plot_arrow_3d(np.array(Oxyz), np.array(Oxyz) + z, color='black', cone_size=0.1, arrow_length_ratio=0.05))

        if xyz_label:
            # Add labels for XYZ axes - positioning might need adjustment based on the scale
            max_range = np.max(np.abs(xyz))
            fig.add_trace(go.Scatter3d(
                x=[x0 + x[0] * 1.1], y=[y0 + x[1] * 1.1], z=[z0 + x[2] * 1.1],
                mode='text', text=['X'], textposition='middle right', showlegend=False,
                textfont=dict(color='black', size=15)
            ))
            fig.add_trace(go.Scatter3d(
                x=[x0 + y[0] * 1.1], y=[y0 + y[1] * 1.1], z=[z0 + y[2] * 1.1],
                mode='text', text=['Y'], textposition='bottom center', showlegend=False,
                textfont=dict(color='black', size=15)
            ))
            fig.add_trace(go.Scatter3d(
                x=[x0 + z[0] * 1.1], y=[y0 + z[1] * 1.1], z=[z0 + z[2] * 1.1],
                mode='text', text=['Z'], textposition='top center', showlegend=False,
                textfont=dict(color='black', size=15)
            ))

    i0, j0, k0 = Oijk
    if len(ijk):
        ijk = np.asarray(ijk)
        if len(point):
            point = np.asarray(point)
            scale = np.abs(point.max(axis=0) - point.min(axis=0))
            scale = scale.max()
            ijk = ijk * scale/2
        i, j, k = ijk
        # i-axis
        fig.add_traces(plot_arrow_3d(np.array(Oijk), np.array(Oijk) + i, color='red', arrow_length_ratio=0.01, cone_size=0.01))
        # j-axis
        fig.add_traces(plot_arrow_3d(np.array(Oijk), np.array(Oijk) + j, color='green', arrow_length_ratio=0.01, cone_size=0.01))
        # k-axis
        fig.add_traces(plot_arrow_3d(np.array(Oijk), np.array(Oijk) + k, color='blue', arrow_length_ratio=0.01, cone_size=0.01))

        if ijk_label:
             # Add labels for ijk axes
            fig.add_trace(go.Scatter3d(
                x=[i0 + i[0] * 1.1], y=[j0 + i[1] * 1.1], z=[k0 + i[2] * 1.1],
                mode='text', text=['i'], textposition='middle right', showlegend=False, textfont=dict(color='red', size=15)
            ))
            fig.add_trace(go.Scatter3d(
                x=[i0 + j[0] * 1.1], y=[j0 + j[1] * 1.1], z=[k0 + j[2] * 1.1],
                mode='text', text=['j'], textposition='bottom center', showlegend=False, textfont=dict(color='green', size=15)
            ))
            fig.add_trace(go.Scatter3d(
                x=[i0 + k[0] * 1.1], y=[j0 + k[1] * 1.1], z=[k0 + k[2] * 1.1],
                mode='text', text=['k'], textposition='top center', showlegend=False, textfont=dict(color='blue', size=15)
            ))

    if len(point):
        point = np.asarray(point)
        # Add tooltips for points
        if point_label is None:
            hover_text = [f'({p[0]}, {p[1]}, {p[2]})' for p in point]
        else:
            hover_text = [f'{p_label}: ({p[0]}, {p[1]}, {p[2]})' for p, p_label in zip(point, point_label)]
        fig.add_trace(go.Scatter3d(
            x=point[:, 0],
            y=point[:, 1],
            z=point[:, 2],
            mode='markers',
            marker=dict(size=8, color='yellow'),
            name='Data Points', # Legend name
            hoverinfo='text', # Show tooltip on hover
            hovertext=hover_text # Custom tooltip text
        ))

        for xp, yp, zp in point:
            if vector:
                # Use plot_arrow_3d to draw vectors with arrowheads
                fig.add_traces(plot_arrow_3d(np.array(Oxyz), np.array([xp, yp, zp]), color='yellow', arrow_length_ratio=0.1, cone_size=0.05))

            if proj_lines:
                # Projection lines to XY plane
                fig.add_trace(go.Scatter3d(
                    x=[xp, xp], y=[yp, yp], z=[z0, zp],
                    mode='lines', line=dict(color='grey', dash='dot'), showlegend=False, hoverinfo='none'
                ))
                fig.add_trace(go.Scatter3d(
                    x=[x0, xp], y=[yp, yp], z=[z0, z0],
                    mode='lines', line=dict(color='grey', dash='dot'), showlegend=False, hoverinfo='none'
                ))
                fig.add_trace(go.Scatter3d(
                    x=[xp, xp], y=[y0, yp], z=[z0, z0],
                    mode='lines', line=dict(color='grey', dash='dot'), showlegend=False, hoverinfo='none'
                ))

    camera = dict(
        up=dict(x=0, y=2, z=0),
        eye=dict(x=2., y=0., z=0.)
    )

    fig.update_layout(
        #title='3D Coordinate Systems and Data Points', # Add title
        scene=dict(
            xaxis=dict(title='X', showgrid=True, showbackground=True, zeroline=True),
            yaxis=dict(title='Y', showgrid=True, showbackground=True, zeroline=True),
            zaxis=dict(title='Z', showgrid=True, showbackground=True, zeroline=True, autorange="reversed"),
            camera=camera,
            aspectmode='data' # or 'data' or 'auto'
        ),
        #margin=dict(l=0, r=0, b=0, t=40), # Adjust margin for title
        hovermode='closest' # Improve hover behavior
    )

    return fig

## Leitura do arquivo e pré-processamento

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/BMClab/BMC/refs/heads/master/data/walk1.trc', skiprows=3, sep='\t')
cols = list(data.columns)
cols = [col.replace('.', '') for col in cols]
for i in range(2, len(cols)-3, 3):
    marca = cols[i]
    cols[i] = marca + 'X'
    cols[i+1] = marca + 'Y'
    cols[i+2] = marca + 'Z'

cols = [c.upper() for c in cols]
data.columns = cols
data = data.drop(0, axis=0)
data = data.drop(data.columns[[0, -1]], axis=1)
data = data.astype(float)
data.iloc[:, 1:] = data.iloc[:, 1:].values / 1000  # mm to m
display(data)
display(data.columns)

,TIME,RASISX,RASISY,RASISZ,LASISX,LASISY,LASISZ,RPSISX,RPSISY,RPSISZ,...,RMT2Z,LKNEEMEDIALX,LKNEEMEDIALY,LKNEEMEDIALZ,LANKLEMEDIALX,LANKLEMEDIALY,LANKLEMEDIALZ,LMT2X,LMT2Y,LMT2Z
1,0.000,-0.148018,1.039826,-0.479703,-0.165592,1.043436,-0.249120,-0.342912,1.070820,-0.409471,...,-0.412118,-0.062734,0.545297,-0.324152,-0.236939,0.148704,-0.333784,-0.150985,0.051092,-0.261883
2,0.007,-0.141058,1.038861,-0.479913,-0.158079,1.042954,-0.249257,-0.336116,1.070181,-0.409000,...,-0.412210,-0.053445,0.546187,-0.324160,-0.213441,0.143997,-0.333000,-0.123441,0.050214,-0.260259
3,0.013,-0.134332,1.037895,-0.479989,-0.150414,1.042240,-0.249171,-0.329322,1.069535,-0.408503,...,-0.412325,-0.044440,0.546737,-0.324199,-0.189872,0.139392,-0.331940,-0.095634,0.049809,-0.259605
4,0.020,-0.127682,1.036866,-0.480055,-0.142959,1.041588,-0.249026,-0.322433,1.068873,-0.407999,...,-0.412357,-0.035515,0.546795,-0.323374,-0.166322,0.134914,-0.330772,-0.068500,0.048900,-0.258710
5,0.027,-0.120731,1.035838,-0.479962,-0.135651,1.041068,-0.248977,-0.315505,1.068227,-0.407452,...,-0.412444,-0.026123,0.546789,-0.323470,-0.142575,0.130622,-0.329480,-0.041296,0.048466,-0.257679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2.393,2.669729,1.013948,-0.443166,2.679452,1.018445,-0.210724,2.486804,1.050265,-0.350918,...,-0.365184,2.759086,0.523217,-0.321459,2.908786,0.116326,-0.302856,3.050601,0.100390,-0.247426
361,2.400,2.678802,1.013561,-0.442502,2.689003,1.018013,-0.209994,2.496053,1.049709,-0.349817,...,-0.365177,2.768020,0.522362,-0.321177,2.913835,0.114138,-0.303386,3.055600,0.097254,-0.248036
362,2.407,2.688065,1.013247,-0.441769,2.698695,1.017800,-0.209276,2.505448,1.049106,-0.348723,...,-0.365195,2.777018,0.521624,-0.320795,2.917893,0.111588,-0.303889,3.059447,0.093263,-0.248779
363,2.413,2.697386,1.012931,-0.441014,2.708502,1.017577,-0.208479,2.515101,1.048503,-0.347683,...,-0.365259,2.786064,0.520972,-0.320384,2.920929,0.108704,-0.304379,3.062302,0.088656,-0.250159


Index(['TIME', 'RASISX', 'RASISY', 'RASISZ', 'LASISX', 'LASISY', 'LASISZ',
       'RPSISX', 'RPSISY', 'RPSISZ', 'LPSISX', 'LPSISY', 'LPSISZ',
       'LILIACCRESTX', 'LILIACCRESTY', 'LILIACCRESTZ', 'RILIACCRESTX',
       'RILIACCRESTY', 'RILIACCRESTZ', 'RGTRX', 'RGTRY', 'RGTRZ', 'RKNEEX',
       'RKNEEY', 'RKNEEZ', 'RHFX', 'RHFY', 'RHFZ', 'RTTX', 'RTTY', 'RTTZ',
       'RANKLEX', 'RANKLEY', 'RANKLEZ', 'RHEELX', 'RHEELY', 'RHEELZ', 'RMT1X',
       'RMT1Y', 'RMT1Z', 'RMT5X', 'RMT5Y', 'RMT5Z', 'LGTRX', 'LGTRY', 'LGTRZ',
       'LKNEEX', 'LKNEEY', 'LKNEEZ', 'LHFX', 'LHFY', 'LHFZ', 'LTTX', 'LTTY',
       'LTTZ', 'LANKLEX', 'LANKLEY', 'LANKLEZ', 'LHEELX', 'LHEELY', 'LHEELZ',
       'LMT1X', 'LMT1Y', 'LMT1Z', 'LMT5X', 'LMT5Y', 'LMT5Z', 'RKNEEMEDIALX',
       'RKNEEMEDIALY', 'RKNEEMEDIALZ', 'RANKLEMEDIALX', 'RANKLEMEDIALY',
       'RANKLEMEDIALZ', 'RMT2X', 'RMT2Y', 'RMT2Z', 'LKNEEMEDIALX',
       'LKNEEMEDIALY', 'LKNEEMEDIALZ', 'LANKLEMEDIALX', 'LANKLEMEDIALY',
       'LANKLEMEDIALZ', 'LMT2X', 

## Pelve

**Pelvic anatomical planes**
- **pelvic quasi-transverse plane**: this plane is defined by **rasis**, **lasis** and the point midway between **rpsis** and **lpsis** (**Op**).  
- **pelvic quasi-coronal plane**: this plane is orthogonal to the **pelvic quasi-transverse plane** and containing both **rasis** and **lasis**.  
- **pelvic quasi-sagittal plane**: this plane is mutually perpendicular to both **quasi-transverse** and **quasi-coronal plane of the pelvis**.  

**Pelvic anatomical frame**
- **Oa**: the **Oa** point defines the origin of the anatomical frame of the pelvic segment **xpypzp**; this point is the midpoint between the **rasis** and **lasis**.
- **zp-axis**: this axis is oriented along the line passing through the **rasis** and **lasis** with its positive direction pointing right.  
- **xp-axis**: this axis lies in the **pelvic quasi-transverse plane** and, is perpendicular to the **zp-axis**, its positive direction is anterior.  
- **yp-axis**: this axis is mutually perpendicular to both the **xp-axis** and the **zp-axis**, and is pointing upwards.  

### Seleção das marcas necessárias

In [ ]:
RASIS  = data[['RASISX', 'RASISY', 'RASISZ']].values
LASIS  = data[['LASISX', 'LASISY', 'LASISZ']].values
RPSIS  = data[['RPSISX', 'RPSISY', 'RPSISZ']].values
LPSIS  = data[['LPSISX', 'LPSISY', 'LPSISZ']].values

Op = (RASIS + LASIS) / 2

### $\hat{e_3}$

In [ ]:
v3 = RASIS - LASIS
e3p = v3 / np.linalg.norm(v3, axis=1, keepdims=True)

### $\hat{e_2}$

In [ ]:
mPSIS = (RPSIS + LPSIS) / 2
v2 =  np.cross(LASIS - mPSIS, RASIS - mPSIS)
e2p = v2 / np.linalg.norm(v2, axis=1, keepdims=True)

### $\hat{e_1}$

In [ ]:
e1p = np.cross(e3p, e2p)

### Verificar normas

In [ ]:
print(np.all(np.isclose(np.linalg.norm(e1p, axis=1), 1)))
print(np.all(np.isclose(np.linalg.norm(e2p, axis=1), 1)))
print(np.all(np.isclose(np.linalg.norm(e3p, axis=1), 1)))

True
True
True


### Verificar ortogonalidade

In [ ]:
print(np.all(np.isclose(np.sum(e1p*e2p, axis=1), 0)))
print(np.all(np.isclose(np.sum(e1p*e3p, axis=1), 0)))
print(np.all(np.isclose(np.sum(e2p*e3p, axis=1), 0)))

True
True
True


### Visualizar base

In [ ]:
markers = np.vstack((RASIS[0], LASIS[0], RPSIS[0], LPSIS[0]))
basis = np.vstack((e1p[0], e2p[0], e3p[0]))

fig = plot_ccs(Oijk=Op[0], ijk=basis, point=markers, point_label=['RASIS','LASIS','RPSIS','LPSIS'], proj_lines=False, vector=False)
fig.show()

## Pé

**Foot anatomical plane**
- **foot quasi-transverse plane**: this plane is defined by **ca**, **fm** and **vm**.
- **foot quasi-sagittal plane**: this plane is orthogonal to the foot quasi-transverse plane and contains both **ca** and **sm**.  
- **foot quasi-coronal plane**: this plane is mutually perpendicular to the foot quasi-transverse and quasi-sagittal plane.  

**Foot anatomical frame**  
- **Of**: the **Of** point defines the origin of the anatomical frame of the foot segment **xfyfzf**. this point is **ca**.  
- **yf-axis**: this axis is defined by the intersection between the foot quasi-coronal and quasi-sagittal planes with positive direction upwards.  
- **zf-axis**: this axis is lying in the foot quasi-transverse plane and is perpendicular to the **yf-axis**, with positive direction pointing right.  
- **xf-axis**: this axis is mutually perpendicular to the **yf-axis** and the **zf-axis** and is pointing to the anterior.

In [ ]:
CA = data[['RHEELX', 'RHEELY', 'RHEELZ']].values
FM = data[['RMT1X', 'RMT1Y', 'RMT1Z']].values
SM = data[['RMT2X', 'RMT2Y', 'RMT2Z']].values
VM = data[['RMT5X', 'RMT5Y', 'RMT5Z']].values

Of = CA

In [ ]:
v2 =  np.cross(FM - CA, VM - CA)
e2f = v2 / np.linalg.norm(v2, axis=1, keepdims=True)

In [ ]:
v3 =  np.cross(v2, SM - CA)
e3f = v3 / np.linalg.norm(v3, axis=1, keepdims=True)

In [ ]:
e1f = np.cross(e3f, e2f)

In [ ]:
markers = np.vstack((CA[0], FM[0], SM[0], VM[0]))
basis = np.vstack((e1f[0], e2f[0], e3f[0]))

fig = plot_ccs(Oijk=Of[0], ijk=basis, point=markers, point_label=['CA','FM','SM','VM'], proj_lines=False, vector=False)
fig.show()

## Perna

In [ ]:
HF = data[['RHFX', 'RHFY', 'RHFZ']].values
TT = data[['RTTX', 'RTTY', 'RTTZ']].values
MM = data[['RANKLEMEDIALX', 'RANKLEMEDIALY', 'RANKLEMEDIALZ']].values
ML = data[['RANKLEX', 'RANKLEY', 'RANKLEZ']].values

Os = (MM + ML) / 2

In [ ]:
v1 =  np.cross(MM - HF, ML - HF)
e1s = v1 / np.linalg.norm(v1, axis=1, keepdims=True)

In [ ]:
v3 =  np.cross(TT - Os, e1s)
e3s = v3 / np.linalg.norm(v3, axis=1, keepdims=True)

In [ ]:
e2s = np.cross(e1s, e3s)

In [ ]:
markers = np.vstack((HF[0], TT[0], MM[0], ML[0]))
basis = np.vstack((e1s[0], e2s[0], e3s[0]))

fig = plot_ccs(Oijk=Os[0], ijk=basis, point=markers, point_label=['HF','TT','MM','ML'], proj_lines=False, vector=False)
fig.show()

## Coxa

Para determinar o sistema de coordenadas anatômico usando a convenção VAKHUM é necessário a posição da cabeça do fêmur (FH), que é o centro articular do quadril (HJC). Obviamente não é possivel colocar um marcador na cabeça do fêmur para rastrear sua posição porque é uma região dentro do corpo.  
O que usualmente se faz é usar algum método indireto para estimar esta posição, por exemplo a partir de equações de regressão que relacionam as dimensões da pelve com a posição da cabeça do fêmur. Estas equações de regressão são geralmente obtidas de estudo de uma amostra da população e é utilizado algum outro método (por exemplo, raio-x).

Para estimar a posição da cabeça do fêmur, vamos utilizar o método proposto por Bell e Brand (1989):  
> RFH(RHJC) = Op + dASI*[-0.19, -0.3,  0.36]  

onde Op é a origem do sistema de coordenadas da pelve e dASIS é a distância entre as duas ASIS, na ordem [AP, Vert, ML].

In [ ]:
dASIS = np.linalg.norm(RASIS - LASIS, axis=1, keepdims=True)
FH = Op + dASIS*np.repeat(np.array([-0.19, -0.3, -0.36], ndmin=2), Op.shape[0], axis=0)

Vamos checar nossa estimativa do marcador FH plotando junto com o sistema de coordenadas da pelve:

In [ ]:
markers = np.vstack((RASIS[0], LASIS[0], RPSIS[0], LPSIS[0], FH[0]))
basis = np.vstack((e1p[0], e2p[0], e3p[0]))

fig = plot_ccs(Oijk=Op[0], ijk=basis, point=markers, point_label=['RASIS','LASIS','RPSIS','LPSIS', 'FH'], proj_lines=False, vector=False)
fig.show()

In [ ]:
LE = data[['RKNEEX', 'RKNEEY', 'RKNEEZ']].values
ME = data[['RKNEEMEDIALX', 'RKNEEMEDIALY', 'RKNEEMEDIALZ']].values

Ot = (LE + ME) / 2

In [ ]:
v2 = FH - Ot
e2t = v2 / np.linalg.norm(v2, axis=1, keepdims=True)

In [ ]:
v1 = np.cross(LE - Ot, v2)
e1t = v1 / np.linalg.norm(v1, axis=1, keepdims=True)

In [ ]:
e3t = np.cross(e2t, e1t)

In [ ]:
markers = np.vstack((LE[0], ME[0], FH[0]))
basis = np.vstack((e1t[0], e2t[0], e3t[0]))

fig = plot_ccs(Oijk=Ot[0], ijk=basis, point=markers, point_label=['LE','ME', 'FH'], proj_lines=False, vector=False)
fig.show()